<a href="https://colab.research.google.com/github/aryabhanushali/murtylab-experiments/blob/main/2_Layer_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

#MNIST dataset
#preprocess
#dataload

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]) #convert images to pytorch tensors, normalize pixels

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)#load training set
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True) #take 64 at a time, shuffle

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)#load test set
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)#64 at a time but no need to shuffle

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(28*28, 128)  #layer 1 (take in 784, output 128 neurons)
        self.layer2 = nn.Linear(128, 64)     #layer 2 (take in 128, output 64 neurons)
        self.output3 = nn.Linear(64, 10)      #output layer (take in 64, output 10 neurons) digits 0-9

    def forward(self, x):
        x = x.view(-1, 28*28)  #before passing to nn, must make 1D to single col
        x = torch.relu(self.layer1(x))  #fc1 Apply ReLU activation
        x = torch.relu(self.layer2(x))  #fc2 Apply ReLU activation
        x = self.output3(x)
        return x

# create instance of this class to make a model we can use
model = MLP()

In [3]:
criterion = nn.CrossEntropyLoss()  # compare the predicted to the true labels and calculates the loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  #adjusts the weights of a neural network to minimize the loss function


In [4]:
num_epochs = 7  #go through the entire dataset 7 times

for epoch in range(num_epochs): #model to refine its weights multiple times
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)  #forward pass
        loss = criterion(outputs, labels)  #compute loss
        loss.backward()  #backpropagation
        optimizer.step()  #update

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/7], Loss: 0.3533
Epoch [2/7], Loss: 0.2599
Epoch [3/7], Loss: 0.1065
Epoch [4/7], Loss: 0.0229
Epoch [5/7], Loss: 0.0446
Epoch [6/7], Loss: 0.0493
Epoch [7/7], Loss: 0.1480


In [5]:
correct = 0
total = 0
#calculates its accuracy

with torch.no_grad():  #dont need gradient
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


Accuracy: 97.38%
